## **Imports and setup**

In [ ]:
import cv2
import math
import webcolors
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import os
from collections import Counter
from scipy.spatial.distance import cdist

## **Compute Mean Squared Error (MSE) between frames for boundary detection**

In [ ]:
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

## **Semi-Master-Shot Boundary Detection (SBD)**

In [ ]:
def extract_key_frames(video_path, output_folder, threshold=1000):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    prev_frame = None
    frame_id = 0
    key_frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is not None:
            difference = mse(prev_frame, gray)
            if difference > threshold:  # Scene change detected
                frame_path = os.path.join(output_folder, f'frame_{frame_id}.jpg')
                cv2.imwrite(frame_path, frame)
                key_frames.append(frame_path)
                frame_id += 1

        prev_frame = gray.copy()

    cap.release()
    print(f"Extracted {len(key_frames)} key frames based on SBD.")
    return key_frames

## **Compute histogram similarity between two frames**

In [ ]:
def calculate_histogram_difference(frame1, frame2):
    hist1 = cv2.calcHist([frame1], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist2 = cv2.calcHist([frame2], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    return cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)

## **Generate saliency map for a given frame**

In [ ]:
def generate_saliency_map(image_path):
    frame = cv2.imread(image_path)
    saliency = cv2.saliency.StaticSaliencyFineGrained_create()
    success, saliency_map = saliency.computeSaliency(frame)
    saliency_map = (saliency_map * 255).astype(np.uint8)
    return saliency_map

## **Extract dominant colors using k-means clustering**

In [ ]:
def get_color_palette(image_path, n_colors=10):
    img = cv2.imread(image_path)
    saliency_map = generate_saliency_map(image_path)
    masked_img = cv2.bitwise_and(img, img, mask=saliency_map)

    img = cv2.cvtColor(masked_img, cv2.COLOR_BGR2RGB)
    img = img.reshape((-1, 3))
    img = img[np.any(img != [0, 0, 0], axis=1)]  # Remove black pixels

    if img.shape[0] == 0:
        return [], saliency_map

    kmeans = KMeans(n_clusters=min(n_colors, len(img)), random_state=42, n_init=10).fit(img)
    colors = kmeans.cluster_centers_.astype(int)
    counts = Counter(kmeans.labels_)
    total_pixels = sum(counts.values())
    color_percentages = [(colors[i], counts[i] / total_pixels * 100) for i in range(len(colors))]
    color_percentages.sort(key=lambda x: x[1], reverse=True)

    return color_percentages, saliency_map

## **Color Similarity Merging (CSM) Algorithm**

In [ ]:
def merge_similar_colors(color_list, threshold=50):
    if not color_list:
        return []

    colors = np.array([c[0] for c in color_list])
    distances = cdist(colors, colors, metric='euclidean')
    merged_colors = []
    used = set()

    for i in range(len(colors)):
        if i in used:
            continue

        close_colors = [colors[i]]
        for j in range(i + 1, len(colors)):
            if j not in used and distances[i, j] < threshold:
                close_colors.append(colors[j])
                used.add(j)

        merged_colors.append(np.mean(close_colors, axis=0).astype(int))

    return merged_colors

## **Display key frames alongside their saliency maps and color palettes**

In [ ]:
def display_key_frames_with_palettes(frames_folder, key_frames):
    plt.figure(figsize=(15, 7 * len(key_frames)))

    all_colors = []
    valid_frames = 0
    for frame_path in key_frames:
        img = cv2.cvtColor(cv2.imread(frame_path), cv2.COLOR_BGR2RGB)
        colors, saliency_map = get_color_palette(frame_path)

        if not colors:
            continue

        all_colors.extend(colors)

        plt.subplot(len(key_frames), 3, 3 * valid_frames + 1)
        plt.imshow(img)
        plt.title(frame_path.split("/")[-1])
        plt.axis('off')

        plt.subplot(len(key_frames), 3, 3 * valid_frames + 2)
        plt.imshow(saliency_map, cmap='gray')
        plt.title("Saliency Map")
        plt.axis('off')

        plt.subplot(len(key_frames), 3, 3 * valid_frames + 3)
        palette = np.zeros((100, 500, 3), dtype=np.uint8)
        step = 500 // len(colors)

        for j, (color, percent) in enumerate(colors):
            palette[:, j * step:(j + 1) * step] = color
            plt.text(j * step, 110, f"{percent:.2f}%", fontsize=8, ha='center')

        plt.imshow(palette)
        plt.axis('off')
        plt.title("Color Palette with Percentages")
        valid_frames += 1

    plt.tight_layout()
    plt.show()
    # Merge color palettes from all frames
    merged_colors = merge_similar_colors(all_colors)

    # Display final merged color scheme
    plt.figure(figsize=(10, 2))
    final_palette = np.zeros((50, 500, 3), dtype=np.uint8)
    step = 500 // len(merged_colors)

    for i, color in enumerate(merged_colors):
        final_palette[:, i * step:(i + 1) * step] = color

    plt.imshow(final_palette)
    plt.axis('off')
    plt.title("Final Merged Color Scheme")
    plt.show()

## **DRIVER CODE**

In [ ]:
# for every movie you want to work at the same time create new driver with unique folder name for frames.

video_path = 'path_to_your_video'
frames_folder = 'extracted_frames'
key_frames = extract_key_frames(video_path, frames_folder)
display_key_frames_with_palettes(frames_folder, key_frames)


## **Define updated video paths**

In [ ]:
video_paths = {
    "movie 1": "path/movie 1",
    "movie 2": "path/movie 2",
    # for example
    "The Grand Budapest Hotel": "/content/THE GRAND BUDAPEST HOTEL - Official Wolrdwide Trailer HD.mp4"
}


## **Extract and store the frames for each movie in different folder**

In [ ]:

def extract_frame(video_path, frame_number=150):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames == 0:
        print("⚠️ Couldn't read frames from:", video_path)
        return None

    frame_number = min(frame_number, total_frames - 1)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()

    if ret:
        frame_path = "frame.jpg"
        cv2.imwrite(frame_path, frame)
        return frame_path
    else:
        print("❌ Could not extract frame.")
        return None


## **Closest named colour for any palette**

In [ ]:
css3_colors = {
    'black': (0, 0, 0), 'white': (255, 255, 255), 'red': (255, 0, 0), 'lime': (0, 255, 0),
    'blue': (0, 0, 255), 'yellow': (255, 255, 0), 'cyan': (0, 255, 255), 'magenta': (255, 0, 255),
    'silver': (192, 192, 192), 'gray': (128, 128, 128), 'maroon': (128, 0, 0),
    'olive': (128, 128, 0), 'green': (0, 128, 0), 'purple': (128, 0, 128),
    'teal': (0, 128, 128), 'navy': (0, 0, 128)
}

def closest_colour(requested_colour):
    min_distance = float('inf')
    closest_name = None
    for name, rgb in css3_colors.items():
        distance = math.sqrt(sum((rc - sc) ** 2 for rc, sc in zip(rgb, requested_colour)))
        if distance < min_distance:
            min_distance = distance
            closest_name = name
    return closest_name

def get_color_name(rgb_tuple):
    try:
        return webcolors.rgb_to_name(rgb_tuple)
    except ValueError:
        return closest_colour(rgb_tuple)

## **Dominant color name for each movie**

In [ ]:

for title, path in video_paths.items():
    print(f"\n🎞️ Processing: {title}")
    frame_path = extract_frame(path)
    if frame_path:
        color_percentages, _ = get_color_palette(frame_path)
        if color_percentages:
            dominant_rgb = tuple(color_percentages[0][0])
            dominant_name = get_color_name(dominant_rgb)
            print(f"🎨 Dominant RGB: {dominant_rgb}")
            print(f"🟢 Recommended Color Name: {dominant_name}")
        else:
            print("⚠️ No colors found.")
        os.remove(frame_path)
    else:
        print("❌ Failed to extract frame.")


## **Necessary Formatting**

In [ ]:
# HEX formatter
def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

# Safe CSS3 color name detection (no webcolors dependency)
def closest_colour(requested_colour):
    css3_colors = {
        'black': (0, 0, 0), 'white': (255, 255, 255), 'red': (255, 0, 0), 'lime': (0, 255, 0),
        'blue': (0, 0, 255), 'yellow': (255, 255, 0), 'cyan': (0, 255, 255), 'magenta': (255, 0, 255),
        'silver': (192, 192, 192), 'gray': (128, 128, 128), 'maroon': (128, 0, 0),
        'olive': (128, 128, 0), 'green': (0, 128, 0), 'purple': (128, 0, 128),
        'teal': (0, 128, 128), 'navy': (0, 0, 128), 'orange': (255, 165, 0), 'pink': (255, 192, 203)
    }

    min_distance = float('inf')
    closest_name = None
    for name, rgb in css3_colors.items():
        dist = sum((rc - sc) ** 2 for rc, sc in zip(rgb, requested_colour))
        if dist < min_distance:
            min_distance = dist
            closest_name = name
    return closest_name

## **Genre detection from RGB**

In [ ]:
def infer_genre_from_rgb(rgb):
    r, g, b = rgb
    if sum(rgb) < 90:
        return "Sci-Fi, Thriller, Noir"
    elif r > 180 and g < 100 and b < 100:
        return "Romance, Drama"
    elif r > 150 and g > 100 and b < 100:
        return "Adventure, Coming-of-Age"
    elif b > 180 and r < 150:
        return "Fantasy, Mystery"
    elif r > 200 and g > 150 and b > 150:
        return "Comedy, Feel-Good"
    else:
        return "Experimental, Mixed Genre"

# Folder map used in previous dominant color step
movie_frames = {
    "movie 1": "path/movie 1",
    "movie 2": "path/movie 2",
    # for example
    "The Grand Budapest Hotel": "extracted_frames_grand"
}

## **Final step to show genre and suggestions**

In [ ]:
print("\n🎬 Genre Suggestions Based on Dominant Color:\n")
for title, folder in movie_frames.items():
    key_frames = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".jpg")]
    top_colors = []

    for frame_path in key_frames:
        colors, _ = get_color_palette(frame_path)
        if colors:
            top_colors.append(tuple(map(int, colors[0][0])))

    if not top_colors:
        print(f"{title}: ⚠️ No dominant color found.\n")
        continue

    majority_rgb = Counter(top_colors).most_common(1)[0][0]
    majority_hex = rgb_to_hex(majority_rgb)
    color_name = closest_colour(majority_rgb)
    genre = infer_genre_from_rgb(majority_rgb)

    print(f"{title}:")
    print(f"🎨 Dominant RGB: {majority_rgb}")
    print(f"🟢 HEX: {majority_hex}, Color Name: {color_name}")
    print(f"🎬 Suggested Genres: {genre}\n")
